<a href="https://colab.research.google.com/github/Aiadevop/10-socket-chat/blob/main/DS_NL_Clasificaci%C3%B3n_MODELO_LightGBM_Predicci%C3%B3n_calidad_del_vino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
url="https://raw.githubusercontent.com/Aiadevop/training_model_wine_quality/refs/heads/main/data/df_cl_final.csv"
df_cl = pd.read_csv(url)
# Divido los datos para test y entrenamiento.
from sklearn.model_selection import train_test_split

# Dividir el DataFrame completo. Test_size 20,80 y random_state es la semilla
df_train, df_test = train_test_split(
    df_cl, test_size=0.2, random_state=42, stratify=df_cl['quality']
)

# Luego extraer X e y de cada DataFrame
X_train = df_train.drop('quality', axis=1)
y_train = df_train['quality']

X_test = df_test.drop('quality', axis=1)
y_test = df_test['quality']

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Crear modelos con configuración básica
lgbm = LGBMClassifier(random_state=42, verbose=-1)

# Entrenar y predecir - LightGBM
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
y_proba_lgbm = lgbm.predict_proba(X_test)[:,1]

# Evaluar
acc_lgbm = accuracy_score(y_test, y_pred_lgbm)
roc_lgbm = roc_auc_score(y_test, y_proba_lgbm)

# Imprimir resultados
print("📈 RESULTADOS BOOSTING 📈\n")
print(f"LightGBM     - Accuracy: {acc_lgbm:.3f}, ROC-AUC: {roc_lgbm:.3f}")


📈 RESULTADOS BOOSTING 📈

LightGBM     - Accuracy: 0.816, ROC-AUC: 0.890


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

# Modelo base
lgbm = LGBMClassifier(random_state=42)

# Espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10, -1],  # -1 = sin límite
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [15, 31, 63, 127],
    'min_child_samples': [5, 10, 20],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

# RandomizedSearchCV
random_search_lgbm = RandomizedSearchCV(
    lgbm,
    param_distributions=param_dist,
    n_iter=30,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Entrenar
random_search_lgbm.fit(X_train, y_train)

# Mejor modelo
best_lgbm = random_search_lgbm.best_estimator_
y_pred_lgbm = best_lgbm.predict(X_test)
y_proba_lgbm = best_lgbm.predict_proba(X_test)[:, 1]

# Métricas
acc_lgbm = accuracy_score(y_test, y_pred_lgbm)
roc_lgbm = roc_auc_score(y_test, y_proba_lgbm)

# Resultados
print("\n🔧 Mejor configuración de LightGBM:")
print(random_search_lgbm.best_params_)

print(f"\n✅ Resultados LightGBM optimizado:")
print(f"Accuracy: {acc_lgbm:.3f}")
print(f"ROC-AUC:  {roc_lgbm:.3f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits

🔧 Mejor configuración de LightGBM:
{'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'num_leaves': 63, 'n_estimators': 100, 'min_child_samples': 5, 'max_depth': -1, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

✅ Resultados LightGBM optimizado:
Accuracy: 0.816
ROC-AUC:  0.903
